# Evaluate model

## Imports

In [40]:
import pathlib
import re
import tempfile
import shutil

import tensorflow as tf

from tensorflow.keras import models
from tensorflow.keras import metrics
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing import image

## Config

In [2]:
MODEL_PATH = pathlib.Path(
    "/media/cicheck/Extreme Pro/models/mod_1/best_mod_1_meso_net_05_0.11.h5"
)
DATASET_PATH = pathlib.Path("/media/cicheck/Extreme Pro/datasets/modified/mod_1")

IMAGE_SIZE = (256, 256)
BATCH_SIZE = 32

## Utils

In [3]:
_metric_names_to_rephrased_names = {
    "binary_accuracy": "accuracy",
    "auc": "AUC",
    "precision": "precision",
    "recall": "recall",
    "true_positives": "TP",
    "true_negatives": "TN",
    "false_positives": "FP",
    "false_negatives": "FN",
}

def _rephrase_metrics_dict(metrics_dict: dict[str, float]) -> dict[str, float]:
    # Remove sufixes, e.g. auc_3 -> auc
    prefix_pattern = re.compile(r"(?P<metric_prefix>\w+)_(?P<metric_number>\d+)")
    # TODO: ugly comperhension
    normalized_metrics_dict = {
        (prefix_pattern.match(metric).group("metric_prefix") if prefix_pattern.match(metric) else metric): value
        for metric, value in metrics_dict.items()
    }
    return {
        (_metric_names_to_rephrased_names[metric] if metric in _metric_names_to_rephrased_names else metric): value
        for metric, value in normalized_metrics_dict.items()
    }

def _build_single_row(*columns, width):
    row_as_string = "|"
    for column in columns:
        row_as_string += column.ljust(width, " ")
        row_as_string += "|"
    row_as_string +="\n"
    return row_as_string

def print_metrics_dict_as_table(
    metrics_dict: dict[str, float], width: int = 14
):
    """Print metrics dictionay as markdown table.
    
    Args:
        metrics_dict: metrics dictionay (i.e. output of keras evaluate which output type set to dict).
        width: width of 
    """
    first_column_name = " metrics"
    second_column_name = " values"
    table_as_string = ""
    table_as_string += _build_single_row(first_column_name, second_column_name, width=width)
    header_sign = ":---:"
    table_as_string += _build_single_row(header_sign, header_sign, width=width)
    rephrased_metric_dict = _rephrase_metrics_dict(metrics_dict)
    for metric, value in rephrased_metric_dict.items():
        table_as_string += _build_single_row(metric, "{:.5f}".format(value), width=width)
    print(table_as_string)
        

## Setup model

In [4]:
evaluated_model = models.load_model(MODEL_PATH, compile=False)

2021-09-25 19:06:10.396494: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-09-25 19:06:10.476601: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-25 19:06:10.476898: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3060 Laptop GPU computeCapability: 8.6
coreClock: 1.425GHz coreCount: 30 deviceMemorySize: 5.80GiB deviceMemoryBandwidth: 312.97GiB/s
2021-09-25 19:06:10.476913: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-09-25 19:06:10.486393: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2021-09-25 19:06:10.486465: I tensorflow/stream_e

In [5]:
METRICS = [
        metrics.BinaryAccuracy(),
        metrics.AUC(),
        metrics.Precision(),
        metrics.Recall(),
        metrics.TruePositives(),
        metrics.TrueNegatives(),
        metrics.FalsePositives(),
        metrics.FalseNegatives(),
    ]

In [6]:
# Model needs to be compiled again to set targeted metrices
optimizer = optimizers.Adam(
    learning_rate=1e-3,
    epsilon=1e-08
)

evaluated_model.compile(
    optimizer=optimizer,
    loss="binary_crossentropy",
    metrics=METRICS
)

## Evaluate whole dataset

In [8]:
evaluation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    DATASET_PATH.joinpath("test"),
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    label_mode="binary",
    class_names=["fakes", "reals"],
)

Found 468664 files belonging to 2 classes.


In [9]:
metrics_dict = evaluated_model.evaluate(evaluation_dataset, return_dict=True)

2021-09-25 18:17:25.134359: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-09-25 18:17:25.168160: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 3193960000 Hz
2021-09-25 18:17:25.565230: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2021-09-25 18:17:26.261086: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8100
2021-09-25 18:17:27.323722: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11


    3/14646 [..............................] - ETA: 7:46 - loss: 0.0349 - binary_accuracy: 0.9896 - auc: 1.0000 - precision: 0.8889 - recall: 1.0000 - true_positives: 8.0000 - true_negatives: 87.0000 - false_positives: 1.0000 - false_negatives: 0.0000e+00        

2021-09-25 18:17:27.797420: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2021-09-25 18:17:27.894545: I tensorflow/stream_executor/cuda/cuda_blas.cc:1838] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


14646/14646 [==============================] - 505s 34ms/step - loss: 0.1078 - binary_accuracy: 0.9670 - auc: 0.9719 - precision: 0.8469 - recall: 0.8118 - true_positives: 37445.0000 - true_negatives: 415771.0000 - false_positives: 6768.0000 - false_negatives: 8680.0000


In [16]:
print_metrics_dict_as_table(metrics_dict)

| metrics      | values       |
|:---:         |:---:         |
|loss          |0.10779       |
|accuracy      |0.96704       |
|AUC           |0.97193       |
|precision     |0.84692       |
|recall        |0.81182       |
|TP            |37445.00000   |
|TN            |415771.00000  |
|FP            |6768.00000    |
|FN            |8680.00000    |



In [11]:
metrics_dict

{'loss': 0.15628106892108917,
 'binary_accuracy': 0.9542379975318909,
 'auc': 0.9427394866943359,
 'precision': 0.7974686622619629,
 'recall': 0.7171598672866821,
 'true_positives': 33079.0,
 'true_negatives': 414138.0,
 'false_positives': 8401.0,
 'false_negatives': 13046.0}

In [12]:
metrics_dict

{'loss': 0.15628106892108917,
 'binary_accuracy': 0.9542379975318909,
 'auc': 0.9427394866943359,
 'precision': 0.7974686622619629,
 'recall': 0.7171598672866821,
 'true_positives': 33079.0,
 'true_negatives': 414138.0,
 'false_positives': 8401.0,
 'false_negatives': 13046.0}

In [13]:
# Fajnie było widać że jedyne co się zmieniło to niektóre true positives przeszły na false negatives

## Evaluate modifications separately

In [35]:
modifications_dir = DATASET_PATH / "test" / "reals"

In [47]:
for modification_dir in modifications_dir.iterdir():
    no_mod_images = len(list(modification_dir.iterdir()))
    # Copy selected modification dir into temp dirsince keras method
    # image_dataset_from_directory requires images to lay in subdirectory
    with tempfile.TemporaryDirectory() as tmp_dir_path:
        shutil.copytree(
            modification_dir, pathlib.Path(tmp_dir_path).joinpath(modification_dir.name)
        )
        modifications_ds =  tf.keras.preprocessing.image_dataset_from_directory(
            tmp_dir_path,
            labels=list(1 for _ in range(no_mod_images)),
            batch_size=BATCH_SIZE,
            image_size=IMAGE_SIZE,
        )
        metrics_dict = evaluated_model.evaluate(modifications_ds, return_dict=True)
        print(modification_dir.name)
        print_metrics_dict_as_table(metrics_dict)

Found 5766 files belonging to 1 classes.
181/181 [==============================] - 4s 22ms/step - loss: 0.0913 - binary_accuracy: 0.9591 - auc: 0.0000e+00 - precision: 1.0000 - recall: 0.9591 - true_positives: 5530.0000 - true_negatives: 0.0000e+00 - false_positives: 0.0000e+00 - false_negatives: 236.0000
clahe_8_8_2.0
| metrics      | values       |
|:---:         |:---:         |
|loss          |0.09134       |
|accuracy      |0.95907       |
|AUC           |0.00000       |
|precision     |1.00000       |
|recall        |0.95907       |
|TP            |5530.00000    |
|TN            |0.00000       |
|FP            |0.00000       |
|FN            |236.00000     |

Found 2883 files belonging to 1 classes.
91/91 [==============================] - 2s 23ms/step - loss: 0.4034 - binary_accuracy: 0.8363 - auc: 0.0000e+00 - precision: 1.0000 - recall: 0.8363 - true_positives: 2411.0000 - true_negatives: 0.0000e+00 - false_positives: 0.0000e+00 - false_negatives: 472.0000
gamma_correction_0.